In [2]:
from datasets import load_dataset, Dataset
from bs4 import BeautifulSoup
import re

/home/rodoggx/Documents/iResearch-2024/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Uploading Noisy Dataset

In [3]:
dataset = load_dataset('DATA')

text=[]

for i in dataset['train']:
    xml = i['audio']['path'].replace('.wav', '.xml')
    transcript = []
    with open(xml, 'r') as f:
        content = f.read()
        Bs_data = BeautifulSoup(content, "xml")
        b_unique = Bs_data.find_all('text')
        for j in b_unique:
            transcript.append(j.text)

    text.append(transcript)

for j in range(len(text)):
    for i in range(len(text[j])):
        transcription = text[j][i]
        transcription = re.sub(r'\[.*?\]', '', transcription)
        transcription = transcription.replace('[', '').replace(']', '')

        transcription = re.sub(r'\s+', ' ', transcription)

        text[j][i] = transcription
    

dataset['train'] = dataset['train'].add_column(name='text', column=text)

for i in range(len(text)):
    transcription = ''

    for j in text[i]:
        transcription += j + ' '

    if transcription.endswith(' '):
        transcription = transcription[:-1]

    text[i] = transcription

dataset['train'] = dataset['train'].add_column(name='text_Str', column=text)

dataset = dataset['train'].train_test_split(test_size=0.2)

dataset.push_to_hub('rodoggx/ATCO2-ASR-1h')

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.68s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/rodoggx/ATCO2-ASR-1h/commit/8d0041ff9e13d9f166e75a819fcce95f9e45a5d3', commit_message='Upload dataset', commit_description='', oid='8d0041ff9e13d9f166e75a819fcce95f9e45a5d3', pr_url=None, pr_revision=None, pr_num=None)